# How to save lightcurve data locally?

Saving lightcurve data helps reduce the load on any APIs providing it. There are many ways to save data locally, but some basic methods like csv files, fits files, and pickle serialization ("pickling") will be covered in this tutorial.

## Example: Saving a lightcurve as a csv file

The LightCurve class features a built in `to_csv` method that automatically generates a csv file.

First, obtain a random Kepler Target Pixel File from the MAST data archive and convert it into a lightcurve:

In [1]:
from lightkurve import KeplerTargetPixelFile, KeplerLightCurve
# Open a Kepler Target Pixel File
tpf = KeplerTargetPixelFile.from_archive(6922244, quarter=4)

# Convert the target pixel file into a light curve
lc = tpf.to_lightcurve(aperture_mask=tpf.pipeline_mask)

Let's confirm we have a lightcurve by checking the metadata:

In [5]:
lc.mission

'Kepler'

In [6]:
lc.quarter

4

Now, we can save the lightcurve using the `to_csv` method:

In [7]:
lc.to_csv('K6922244q4.csv')

The file name can be any string with the `.csv` extension, but specifying the Kepler ID and quarter help identify its contents.

If no file name is specified, it returns a csv-formatted string.

## Example: Saving a light curve as a fits file

Similar to csv files, the LightCurve class also has a built in `to_fits` method that automatically generates a fits file.

First, obtain a random Kepler Light Curve File from the Mast data archive and separate the SAP flux:

In [8]:
from lightkurve import KeplerLightCurveFile, KeplerLightCurve
lcf = KeplerLightCurveFile.from_archive(6679295, quarter=4)
sapflux = lcf.SAP_FLUX

Now, edit the lightcurve by removing outliers:

In [9]:
no_outlier = sapflux.remove_outliers()

An edited lightcurve can be effectively saved by using the `to_fits()` method:

In [25]:
no_outlier.to_fits('K6679295q4.fits', overwrite=True)

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f7214e71898>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f7214e71ef0>]

Any string can be used as the file name, but specifiying the Kepler ID and quarter help identify the contents.

If no file name is specified, it returns an `astropy.io.fits` object.

We can now open the fits file and confirm the contents:

In [11]:
from astropy.io import fits
fits_file = fits.open('K6679295q4.fits')
print(type(fits_file))

<class 'astropy.io.fits.hdu.hdulist.HDUList'>


We can also check the header of the first HDU object:

In [12]:
print(fits_file[0].header)

SIMPLE  =                    T / conforms to FITS standards                     BITPIX  =                    8 / array data type                                NAXIS   =                    0 / number of array dimensions                     EXTEND  =                    T / file contains extensions                       NEXTEND =                    2 / number of standard extensions                  EXTNAME = 'PRIMARY '           / name of extension                              EXTVER  =                    1 / extension version number (not format version)  ORIGIN  = 'Unofficial data product' / institution responsible for file          DATE    = '2018-10-13'         / file creation date.                            CREATOR = 'lightkurve'         / pipeline job and program used t                TELESCOP= 'KEPLER  '           / telescope                                      INSTRUME= 'Kepler Photometer'  / detector type                                  OBJECT  = '6679295 '           / string 

## Example: Pickling a lightcurve and saving it in a file

First, obtain a random Kepler Light Curve File from the MAST data archive and separate the fluxes:

In [13]:
from lightkurve import KeplerLightCurveFile
lcf = KeplerLightCurveFile.from_archive(757076, quarter=3)
pdcsap = lcf.PDCSAP_FLUX
sapflux = lcf.SAP_FLUX

Next, pickle the fluxes and save them in files:

In [14]:
import pickle
with open('K757076q3pdc.pickle', 'wb') as pickle_file:
    pickle.dump(pdcsap, pickle_file)
with open('K757076q3sap.pickle', 'wb') as pickle_file:
    pickle.dump(sapflux, pickle_file)

On line 2, we open a file called `K757076q3pdc.pickle` in binary mode located in the current working directory. Line 3 pickles the PDCSAP flux data from `757076`'s 3rd quarter and saves it inside the open file, while lines 4 and 5 do the same for the SAP flux data. Files opened using `with` statements are closed automatically.

The file names can be any string, but if we use generic file names like `lcf`, there's a chance that it may get overwritten later, or that the data won't be identifiable without unpickling.

Once the pickled data is dumped to a file, the program can safely finish while the data in the file will remain saved.

To access the saved data, the file must be reopened in binary mode and the data within unpickled:

In [15]:
with open('K757076q3pdc.pickle', 'rb') as pickle_file:
    pdc = pickle.load(pickle_file)

Now, we can access the flux data from the variable directly:

In [16]:
print(type(pdc))

<class 'lightkurve.lightcurve.KeplerLightCurve'>


In [17]:
pdc

KeplerLightCurve(ID: 757076)

In [18]:
pdc.mission

'Kepler'

In [19]:
pdc.quarter

3

In [24]:
pdc.flux

array([      nan, 301526.7 , 301533.7 , ..., 301663.22, 301678.34,
       301723.8 ], dtype=float32)